In [90]:
import tkinter as tk
import tkinter.messagebox as messagebox
import pygubu
import pickle

In [91]:
class EditWindow(pygubu.TkApplication):

    def save_func(self):
        self.selected_tank.chapters[self.edited_value] = \
            self.textbox.get('1.0', tk.END)

    def exit_func(self):
        pickle.dump(self.tanks, open('tank_data/tanksGUI.p', 'wb'))
        self.mainwindow.destroy()

    def onselect_tank(self, evt):
        w = evt.widget
        try:
            index = int(w.curselection()[0])

            value = w.get(index)
            self.textbox.delete('1.0', tk.END)
            tank = self.tanks[self.aliases[value]]

            names = [len(value) for value in tank.chapters.keys()]
            # print(names)
            self.listbox_info.delete(0, tk.END)
            self.listbox_info.configure(width=max(names))

            for k, v in enumerate(tank.chapters.keys()):
                self.listbox_info.insert(k+1, v)

            self.selected_tank = tank
            al = self.selected_tank.alias if self.selected_tank.alias else self.selected_tank.name
            self.title.set("Tank: {}, Value: ".format(al))
            
        except IndexError:
            print('pff')

    def onselect_info(self, event):
        w = event.widget

        try:
            index = int(w.curselection()[0])

            value = w.get(index)
            self.textbox.delete('1.0', tk.END)
            self.edited_value = value
            self.textbox.insert(
                tk.END, self.selected_tank.chapters[self.edited_value])
            al = self.selected_tank.alias if self.selected_tank.alias else self.selected_tank.name
            self.title.set("Tank: {}, Value: {}".format(al, value))
        except IndexError:
            pass

    def __init__(self, tanks, aliases):
        self.tanks = tanks
        self.aliases = aliases

        self.selected_tank = None

        self.builder = builder = pygubu.Builder()

        builder.add_from_file('GUIs/edit_window.ui')

        self.mainwindow = builder.get_object('edit_window')
        builder.connect_callbacks(self)
        self.listbox_tanks = builder.get_object('listbox_tanks')
        names = [len(tank.alias if tank.alias else tank.name)
                 for tank in tanks.values()]
        self.listbox_tanks.configure(width=max(names))

        for k, v in enumerate(self.tanks.values()):
            if v.alias:
                self.listbox_tanks.insert(k+1, v.alias)
            else:
                self.listbox_tanks.insert(k+1, v.name)

        self.listbox_info = builder.get_object('listbox_info')
        self.listbox_tanks.bind('<<ListboxSelect>>', self.onselect_tank)
        self.listbox_info.bind('<<ListboxSelect>>', self.onselect_info)

        self.textbox = builder.get_object('textbox_info')
        self.title = tk.StringVar()
        self.title.set('Tank: , Value: ')
        label = self.builder.get_object('Label_1')
        label.configure(textvariable=self.title)

In [95]:
class ListWindow(pygubu.TkApplication):

    
    def onselect(self, event):
        w = event.widget
        try:
            index = int(w.curselection()[0])
            value = w.get(index)
            self.textbox.delete('1.0', tk.END)
#             print(value)
            tank = self.tanks[self.aliases[value]]
            tank_string = tank.show_all()
            self.textbox.insert(tk.END, tank_string)
        except IndexError:
            pass
    
    def back_func(self):
        self.mainwindow.destroy()

    def __init__(self, tanks, aliases):
        self.tanks = tanks
        self.aliases = aliases

        self.builder = builder = pygubu.Builder()

        builder.add_from_file('GUIs/list_window.ui')

        self.mainwindow = builder.get_object('list_of_tanks')

        builder.connect_callbacks(self)

        self.textbox = builder.get_object('text_tank_specs')
        self.listbox = builder.get_object('listbox_tanks')
        
        self.listbox.bind('<<ListboxSelect>>', self.onselect)

        names = [len(tank.alias if tank.alias else tank.name)
                 for tank in tanks.values()]
        self.listbox.configure(width=max(names))

        for k, v in enumerate(self.tanks.values()):
            if v.alias:
                self.listbox.insert(k+1, v.alias)
            else:
                self.listbox.insert(k+1, v.name)

In [99]:
class MainWindow(pygubu.TkApplication):

    def exit_func(self):
        self.root.destroy()

    def edit_func(self):
        edit_window = EditWindow(self.tanks, self.aliases)

    def add_func(self):
        pass

    def compare_func(self):
        tk.messagebox.showinfo('Message', 'You clicked Button 2')

    def list_func(self):
        list_window = ListWindow(self.tanks, self.aliases)

    def __init__(self, root):
        self.root = root
        tanks = dict()

        if tanks:= pickle.load(open('tank_data/tanksGUI.p', 'rb')):
#         if tanks:= pickle.load(open('tank_data/tanks.p', 'rb')):
            for t in tanks.values():
                if t.alias:
                    #                     print(t.alias)
                    pass
                else:
                    #                     print(t.name)
                    pass

        self.builder = builder = pygubu.Builder()

        builder.add_from_file('GUIs/mainwindow.ui')

        self.mainwindow = builder.get_object('mainwindow')

#         callbacks = {
#             'add_func': self.add_func,
#             'compare_func': self.compare_func,
#             'list_func': self.list_func
#         }

        builder.connect_callbacks(self)

        self.tanks = tanks

        self.aliases = dict()
        for t in self.tanks.values():
            if t.alias:
                self.aliases[t.alias] = t.name
            else:
                self.aliases[t.name] = t.name
                
        but_add = builder.get_object('button_add')
        but_add.configure(state=tk.DISABLED)
        but_comp = builder.get_object('button_compare')
        but_comp.configure(state=tk.DISABLED)

    def run(self):
        self.mainwindow.mainloop()

In [102]:
root = tk.Tk()
window = MainWindow(root)
window.run()